# **Fetch Data From Exchanges**
This notebook fetches data from major exchanges.

In [18]:
#%% Import Libraries
import pandas as pd
import numpy as np

# The "ccxt" library facilitates connection to various exchanges, including Binance
import ccxt

# The "pycoingecko" library is used to fetch cryptocurrency data from the CoinGecko API
from pycoingecko import CoinGeckoAPI

In [ ]:
#%% Extracting Data from Binance

# List of symbols to fetch data for
symbols = ['BTC/USDT','ETH/USDT','BNB/USDT','SOL/USDT','XRP/USDT','AVAX/USDT']

# Binance API configuration
binance = ccxt.binance()

# List of crypto to fetch data for
timeframe = '1w'  # Timeframe (1 week)

'''
Available timeframes:
'1m' - 1 minute
'1h' - 1 hour
'1d' - 1 day
'1w' - 1 week
'1M' - 1 week
'''

limit = 410  # Number of candles

# Create a list to store the data
all_data = []

# Loop to fetch data for each asset
for symbol in symbols:
    try:
        # Fetch OHLCV data from Binance
        OHLCV = binance.fetch_ohlcv(symbol, timeframe, limit=limit)
        
        print(f"\nSymbol: {symbol}")
        print(f"Number of candles retrieved: {len(OHLCV)}")
        print(f"Date range: {pd.to_datetime(OHLCV[0][0], unit='ms')} to {pd.to_datetime(OHLCV[-1][0], unit='ms')}")
        
        # Process all candles
        for candle in OHLCV:
            data = {
                'symbols': symbol,
                'timestamp': pd.to_datetime(candle[0], unit='ms'),
                'open': candle[1],
                'high': candle[2],
                'low': candle[3],
                'close': candle[4],
                'volume': candle[5]
            }
            all_data.append(data)
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Convert to a DataFrame
ccxt_df = pd.DataFrame(all_data)

# Calculating Average Price
ccxt_df['average_price'] = round((ccxt_df['high'] + ccxt_df['low']) / 2, 2)

# Group by week and calculate the average price
ccxt_df['timestamp'] = pd.to_datetime(ccxt_df['timestamp']).dt.to_period('W')

print(ccxt_df.info())

In [ ]:
#%% Grouping and Sorting Data

# Group by 'symbols' and 'timestamp' to get the average price for each week
avg_df = ccxt_df[['symbols', 'timestamp', 'average_price']]

# Convert symbols to Categorical with specific order from the original crypto DataFrame
avg_df['symbols'] = pd.Categorical(avg_df['symbols'])

# Sort first by symbols (to group all records for each symbol together)
# Then by timestamp (descending) within each symbol group
avg_df = avg_df.sort_values(
    ['symbols', 'timestamp'], 
    ascending=[True, False]
).reset_index(drop=True)

print(avg_df.info())

In [ ]:
#%% Save the DataFrame to a CSV file

avg_df = pd.DataFrame.to_csv(avg_df, 'files/avg_df.csv', index=False)